# Figure 2

In [ ]:
options(warn=-1)

In [ ]:
library_load <- suppressMessages(
    
    suppressWarnings(
        
        list(
        
            # Seurat 
            library(Seurat), 

            # TradeSeq
            library(tradeSeq),

            # Data 
            library(tidyverse), 
            library(data.table), 

            # miloR
            library(miloR), 
            library(ggbeeswarm), 

            # Plotting 
            library(ggplot2), 
            library(patchwork), 
            library(cowplot), 
            library(ComplexHeatmap), 

            # Pyhton compatibility
            library(reticulate)

        )
    )
)

In [ ]:
# Configure reticulate 
# use_condaenv(condaenv="p.3.10.16-FD20200109SPLENO", conda="/nobackup/peer/fdeckert/miniconda3/bin/conda", required=NULL)
# py_config()

In [ ]:
random_seed <- 42
set.seed(random_seed)

In [ ]:
# Set working directory to project root
setwd("/research/peer/fdeckert/FD20200109SPLENO")

In [ ]:
# Source
source("bin/so_pl.R")
source("bin/tradeseq_pp.R")
source("bin/tradeseq_pl.R")
source("bin/reactome.R")

In [ ]:
# Plotting Theme
source("plotting_global.R")
ggplot2::theme_set(theme_global_set(size_select=4)) # From project global source()

# Load reference data 

In [ ]:
tf <- read.table("/research/peer/fdeckert/reference/animaltfdb/Mus_musculus_TF.txt")[[2]]

lr <- CellChat::CellChatDB.mouse[[1]] %>% dplyr::select(pathway_name, ligand, receptor.symbol, receptor.family, annotation) %>% dplyr::filter(annotation %in% c("Cell-Cell Contact", "Secreted Signaling", "ECM-Receptor")) %>% 
    separate_rows(receptor.symbol, sep=", ") %>% dplyr::distinct() %>%
    separate_rows(ligand, sep=", ") %>% dplyr::distinct() %>% 
    dplyr::rename(pathway=pathway_name, receptor=receptor.symbol, family=receptor.family)

# Import scRNAseq data 

In [ ]:
so <- readRDS("data/scRNAseq/object/pp_0.rds")
so <- NormalizeData(so)

In [ ]:
so$sample_group <- factor(so$sample_group, levels=c("Bl6_NaCl_D6", "Bl6_CpG_D6"))

# UMAP erytime WT

In [ ]:
dpt <- read.csv("result/pseudotime/ery_dpt.csv", row.names=1)
so <- AddMetaData(so, dpt)

In [ ]:
fp_1 <- fplot(so, features="dpt_pseudotime", pt_size=0.25, option="F", size_select=4)
fp_1[[1]]$data <- fp_1[[1]]$data[!is.na(fp_1[[1]]$data$features), ]

In [ ]:
pdf("result/figures/figure_2/umap_erytime.pdf", width=3.5, height=2.5)

gridExtra::grid.arrange(
    
    fp_1 %>% egg::set_panel_size(., width=unit(4.5, "cm"), height=unit(4.5, "cm"))

)

dev.off()

# Heatmap of smoothed expression 

In [ ]:
tradeseq_res <- readRDS("result/lineage/tradeseq_res_1.rds")
fitgam <- tradeseq_res[["fitgam"]]
ptpg <- tradeseq_res[["ptpg"]]
condition_vec_1 <- c("Bl6_NaCl_D6", "Bl6_CpG_D6")
color_mat_1 <- color$sample_group[condition_vec_1]

In [ ]:
ptpg_1 <- ptpg[ptpg$contrast==paste0(condition_vec_1, collapse=":"), ]
genes_ptcg_1 <- ptpg[ptpg$ptpg_class %in% c("Canonical"), ][["gene"]]
genes_ptpg_1 <- ptpg[!ptpg$ptpg_class %in% c("Canonical"), ][["gene"]]

## PTCG B (WT) and D6 (WT)

In [ ]:
pt_split_1 <- pt_split(fitgam, genes_ptcg_1, condition_vec_1)

In [ ]:
write.csv(data.frame(genes=pt_split_1[pt_split_1==3] %>% names()), "genes.csv")

In [ ]:
row_genes <- c(
    
    "Meis1", "Klf1", "Tal1", "Gata1", "Gata2", "Epor", "Tfrc", "Alas2", 
    "Mki67", "Pcna", "Top2a", 
    "Pkm", "Cpt1a", "Acadm", "Sdha", "Ndufs1", "G6pdx", "Taldo1", 
    "Icam4"

)

row_genes %in% names(pt_split_1)
row_genes <- row_genes[row_genes %in% names(pt_split_1)]

row_anno <- rowAnnotation(
    
    labels=anno_mark(
        
        at=which(names(pt_split_1) %in% row_genes),
        labels=names(pt_split_1)[which(names(pt_split_1) %in% row_genes)],
        side="left", 
        labels_gp=gpar(fontsize=6), 
        link_width=unit(3, "mm")
    
    )

)

In [ ]:
pt_hm_1 <- pt_hm(fitgam, names(pt_split_1), "Bl6_NaCl_D6", condition_vec_1, 50, color_mat_1[1], row_split=pt_split_1, width=0.75, height=100, fontsize_select=2, use_raster=TRUE)
pt_hm_2 <- pt_hm(fitgam, names(pt_split_1), "Bl6_CpG_D6", condition_vec_1, 50, color_mat_1[2], row_split=pt_split_1, width=0.75, height=100, fontsize_select=2, use_raster=TRUE)

In [ ]:
pdf("result/figures/figure_2/hm_ptcg_wt_ery.pdf", width=3, height=3)

draw(row_anno + pt_hm_1 + pt_hm_2)

dev.off()

## PTPG B (WT) vs D6 (WT)

In [ ]:
pt_split_1 <- pt_split(fitgam, ptpg[ptpg$ptpg_class %in% condition_vec_1[1], ][["gene"]], condition_vec_1[1], n_points=101)

In [ ]:
row_genes <- c(
    
    "Lyl1", "Ifi27", "Usp18", "Oasl1", "Slc3a2", "Slc44a1", "Slc20a1", "Slc25a24"

)

row_genes %in% names(pt_split_1)
row_genes <- row_genes[row_genes %in% names(pt_split_1)]

row_anno <- rowAnnotation(
    
    labels=anno_mark(
        
        at=which(names(pt_split_1) %in% row_genes),
        labels=names(pt_split_1)[which(names(pt_split_1) %in% row_genes)],
        side="left", 
        labels_gp=gpar(fontsize=6), 
        link_width=unit(3, "mm")
    
    )

)

In [ ]:
pt_hm_1 <- pt_hm(fitgam, names(pt_split_1), "Bl6_NaCl_D6", condition_vec_1, 50, color_mat_1[1], row_split=pt_split_1, width=0.75, height=100, fontsize_select=2, use_raster=TRUE)
pt_hm_2 <- pt_hm(fitgam, names(pt_split_1), "Bl6_CpG_D6", condition_vec_1, 50, color_mat_1[2], row_split=pt_split_1, width=0.75, height=100, fontsize_select=2, use_raster=TRUE)

In [ ]:
pdf("result/figures/figure_2/hm_ptpg_ctr_vs_d6_wt.pdf", width=3, height=3)

draw(row_anno + pt_hm_1 + pt_hm_2)

dev.off()

## PTPG D6 (WT) vs B (WT)

In [182]:
pt_split_1 <- pt_split(fitgam, ptpg[ptpg$ptpg_class %in% condition_vec_1[2], ][["gene"]], condition_vec_1[2], n_points=101)

In [186]:
grep("Grb", genes_ptpg_1, value=TRUE)

character(0)

In [ ]:
row_genes <- c(

    "Stat1", "Sox6",
    "Gbp2", "Gbp7", "Iigp1", "Ifi47", "Isg20", "Apol11b", # Interferon  
    "Ccnb1", "Cdc20", "Ube2c", # Cell cycle
    "App", "Hbb-bt", "Hba-a2"

)

row_genes %in% names(pt_split_1)
row_genes <- row_genes[row_genes %in% names(pt_split_1)]

row_anno <- rowAnnotation(
    
    labels=anno_mark(
        
        at=which(names(pt_split_1) %in% row_genes),
        labels=names(pt_split_1)[which(names(pt_split_1) %in% row_genes)],
        side="left", 
        labels_gp=gpar(fontsize=6), 
        link_width=unit(3, "mm")
    
    )

)

In [ ]:
pt_hm_1 <- pt_hm(fitgam, names(pt_split_1), "Bl6_NaCl_D6", condition_vec_1, 50, color_mat_1[1], row_split=pt_split_1, width=0.75, height=100, fontsize_select=2, use_raster=TRUE)
pt_hm_2 <- pt_hm(fitgam, names(pt_split_1), "Bl6_CpG_D6", condition_vec_1, 50, color_mat_1[2], row_split=pt_split_1, width=0.75, height=100, fontsize_select=2, use_raster=TRUE)

In [ ]:
pdf("result/figures/figure_2/hm_ptpg_d6_vs_ctr_wt.pdf", width=3, height=3)

draw(row_anno + pt_hm_1 + pt_hm_2)

dev.off()

# Density celltype plot 

In [ ]:
dp_1 <- ggplot(so[, so$sample_group %in% condition_vec_1[1] & !is.na(so$dpt_pseudotime)]@meta.data, aes(x=na.omit(dpt_pseudotime), y=after_stat(density), fill=celltype_low, alpha=1)) + 
    geom_density(adjust=3.0, size=unit(0.5/2.141959, "pt")) + 
    guides(fill=guide_legend(nrow=2), alpha="none") + 
    scale_fill_manual(values=color$celltype_low) + 
    scale_y_continuous(breaks=c(0, 4, 8), labels=c("0", "4", "8")) + 
    scale_x_continuous(breaks=c(0.0, 0.5, 1.0), labels=c("0.0", "0.5", "1.0"), limits=c(0, 1), expand=c(0, 0)) + 
    theme(legend.position="none")

In [ ]:
pdf("result/figures/figure_2/denp_dpt_ctr.pdf", width=1.5, height=6)

gridExtra::grid.arrange(
    
    dp_1 %>% egg::set_panel_size(., width=unit(18.75, "mm"), height=unit(0.75, "cm")), nrow=1
    
)

dev.off()

In [ ]:
dp_1 <- ggplot(so[, so$sample_group %in% condition_vec_1[2] & !is.na(so$dpt_pseudotime)]@meta.data, aes(x=na.omit(dpt_pseudotime), y=after_stat(density), fill=celltype_low, alpha=1)) + 
    geom_density(adjust=3.0, size=unit(0.5/2.141959, "pt")) + 
    guides(fill=guide_legend(nrow=2), alpha="none") + 
    scale_fill_manual(values=color$celltype_low) + 
    scale_y_continuous(breaks=c(0, 5, 10), labels=c("0", "5", "10")) + 
    scale_x_continuous(breaks=c(0.0, 0.5, 1.0), labels=c("0.0", "0.5", "1.0"), limits=c(0, 1), expand=c(0, 0)) + 
    theme(legend.position="none")

In [ ]:
pdf("result/figures/figure_2/denp_dpt_cpg.pdf", width=1.5, height=6)

gridExtra::grid.arrange(
    
    dp_1 %>% egg::set_panel_size(., width=unit(18.75, "mm"), height=unit(0.75, "cm")), nrow=1
    
)

dev.off()

# Lineplot pseudotime expression 

## Ctr and D6 CpG canonical

In [ ]:
options(repr.plot.width=5, repr.plot.height=5)

genes <- c("Meis1", "Pkm", "Kit", "Gata2", "Gata1", "Klf1", "Epor", "Tfrc", "Gypa", "Alas2")
genes %in% genes_ptcg_1

lp_1 <- lapply(genes, function(i) {
    
    plot_smooth(fitgam, gene=i, point=FALSE, condition_color=color_mat_1, line_size=unit(1.5/2.141959, "pt")) + annotate("text", x=0.5, y=0, label=i, hjust=0.5, vjust=-1, size=2) + theme(legend.position="none")

}
              )

In [ ]:
pdf("result/figures/figure_2/lp_ptcg_wt_ery.pdf", width=1.5, height=8)

gridExtra::grid.arrange(
    
    lp_1[[1]] %>% egg::set_panel_size(., width=unit(2.0, "cm"), height=unit(0.75, "cm")), 
    lp_1[[2]] %>% egg::set_panel_size(., width=unit(2.0, "cm"), height=unit(0.75, "cm")), 
    lp_1[[3]] %>% egg::set_panel_size(., width=unit(2.0, "cm"), height=unit(0.75, "cm")), 
    lp_1[[4]] %>% egg::set_panel_size(., width=unit(2.0, "cm"), height=unit(0.75, "cm")), 
    lp_1[[5]] %>% egg::set_panel_size(., width=unit(2.0, "cm"), height=unit(0.75, "cm")), 
    lp_1[[6]] %>% egg::set_panel_size(., width=unit(2.0, "cm"), height=unit(0.75, "cm")), 
    lp_1[[7]] %>% egg::set_panel_size(., width=unit(2.0, "cm"), height=unit(0.75, "cm")), 
    lp_1[[8]] %>% egg::set_panel_size(., width=unit(2.0, "cm"), height=unit(0.75, "cm")), 
    lp_1[[9]] %>% egg::set_panel_size(., width=unit(2.0, "cm"), height=unit(0.75, "cm")), 
    lp_1[[10]] %>% egg::set_panel_size(., width=unit(2.0, "cm"), height=unit(0.75, "cm")), ncol=1
    
)

dev.off()

## Ctr PTPG upregulated 

In [ ]:
options(repr.plot.width=5, repr.plot.height=5)

genes <- c("Lyl1") 
genes %in% ptpg_1[ptpg_1$ptpg_class=="Bl6_NaCl_D6", ]$gene

lp_1 <- lapply(genes, function(i) {
    
    plot_smooth(fitgam, gene=i, point=FALSE, condition_color=color_mat_1, line_size=unit(1.5/2.141959, "pt")) + annotate("text", x=0.5, y=0, label=i, hjust=0.5, vjust=-1, size=2) + theme(legend.position="none")

}
              )

In [ ]:
pdf("result/figures/figure_2/lp_ptpg_ctr_wt_ery.pdf", width=1.5, height=6)

gridExtra::grid.arrange(
    
    lp_1[[1]] %>% egg::set_panel_size(., width=unit(2.0, "cm"), height=unit(0.75, "cm")), ncol=1
    
)

dev.off()

## D6 PTPG upregulated

In [ ]:
genes <- c("Klf3", "Sox6", "Stat1", "Stat2", "Isg20", "Ifi47", "Gab2", "Gbp7", "Apol11b") 

genes %in% genes_ptpg_1

lp_1 <- lapply(genes, function(i) {
    
    plot_smooth(fitgam, gene=i, point=FALSE, condition_color=color_mat_1, line_size=unit(1.5/2.141959, "pt")) + annotate("text", x=0.5, y=0, label=i, hjust=0.5, vjust=-1, size=2) + theme(legend.position="none") 

}
              )

In [ ]:
pdf("result/figures/figure_2/lp_ptpg_wt_ery.pdf", width=1.5, height=8)

gridExtra::grid.arrange(
    
    lp_1[[1]] %>% egg::set_panel_size(., width=unit(2.0, "cm"), height=unit(0.75, "cm")), 
    lp_1[[2]] %>% egg::set_panel_size(., width=unit(2.0, "cm"), height=unit(0.75, "cm")), 
    lp_1[[3]] %>% egg::set_panel_size(., width=unit(2.0, "cm"), height=unit(0.75, "cm")), 
    lp_1[[4]] %>% egg::set_panel_size(., width=unit(2.0, "cm"), height=unit(0.75, "cm")), 
    lp_1[[5]] %>% egg::set_panel_size(., width=unit(2.0, "cm"), height=unit(0.75, "cm")), 
    lp_1[[6]] %>% egg::set_panel_size(., width=unit(2.0, "cm"), height=unit(0.75, "cm")), 
    lp_1[[7]] %>% egg::set_panel_size(., width=unit(2.0, "cm"), height=unit(0.75, "cm")), 
    lp_1[[8]] %>% egg::set_panel_size(., width=unit(2.0, "cm"), height=unit(0.75, "cm")), 
    lp_1[[9]] %>% egg::set_panel_size(., width=unit(2.0, "cm"), height=unit(0.75, "cm")), ncol=1
    
)

dev.off()

# Pie chart PT gene counts

In [ ]:
ptpg_class_color <- c("Perturbed"="#7f7f7f", "Canonical"="#AEAEAE")
data <- table(ptpg_1 %>% dplyr::mutate(ptpg_class=ifelse(ptpg_class=="Canonical", "Canonical", "Perturbed")) %>% dplyr::pull(ptpg_class)) %>% data.frame() %>% dplyr::rename(group=Var1, count=Freq) %>% dplyr::mutate(group=factor(group, levels=names(ptpg_class_color)))
data
p_pl_1 <- ggplot(data, aes(x="", y=count, fill=group)) +
    geom_col(width=1) +
    coord_polar(theta="y") + 
    scale_fill_manual(values=ptpg_class_color) + 
    ggtitle("Ctr (wt) vs D6 (wt)") + 
    guides(fill=guide_legend(ncol=1, override.aes=list(alpha=1, size=1.5), keywidth=0.25, keyheight=0.25, default.unit="cm"))

In [ ]:
ptpg_class_color <- color$sample_group[c("Bl6_NaCl_D6", "Bl6_CpG_D6")]
data <- table(ptpg_1 %>% dplyr::filter(ptpg_class %in% names(ptpg_class_color)) %>% dplyr::pull(ptpg_class)) %>% data.frame() %>% dplyr::rename(group=Var1, count=Freq) %>% dplyr::mutate(group=factor(group, levels=names(ptpg_class_color)))
data$count %>% sum()
p_pl_2 <- ggplot(data, aes(x="", y=count, fill=group)) +
    geom_col(width=1) +
    coord_polar(theta="y") + 
    scale_fill_manual(values=ptpg_class_color) + 
    ggtitle("Ctr (+/+) vs D1 (+/+)") + 
    guides(fill=guide_legend(ncol=1, override.aes=list(alpha=1, size=1.5), keywidth=0.25, keyheight=0.25, default.unit="cm"))

In [ ]:
pdf("result/figures/figure_2/p_pl_ptg_counts.pdf", width=5, height=2.5)

gridExtra::grid.arrange(
    
    p_pl_1 %>% egg::set_panel_size(., width=unit(2.5, "cm"), height=unit(2.5, "cm")),
    p_pl_2 %>% egg::set_panel_size(., width=unit(2.5, "cm"), height=unit(2.5, "cm")),
    ncol=2
    
)

dev.off()

# Reactome ORA

## PTCG B (WT) and D6 (WT)

In [ ]:
genes <- pt_split(fitgam, genes_ptcg_1, condition_vec_1)

### Split 1

In [ ]:
pt_ora_res <- pt_ora_pa(names(genes[genes==1]), rownames(fitgam))
pt_ora_res <- parse_hierarchy(pt_ora_res) %>% dplyr::mutate(top_level=factor(top_level, levels=names(top_level_color))) %>% dplyr::filter(p.adjust<=0.05) %>% na.omit()

In [ ]:
pdf("result/figures/figure_2/ora_ptcg_split_1_wt.pdf", width=6, height=10)

category_n <- pt_ora_res %>% dplyr::group_by(top_level) %>% dplyr::summarise(n=n()) %>% dplyr::pull(n) /5
(pt_ora_pl_pa(pt_ora_res, color=color_mat_1[1], adj_pval_thr=0.05, range_max=2) + theme_global_set(4)) + ggh4x::force_panelsizes(total_width=unit(2.2, "cm"), rows=unit(category_n, "cm"))

dev.off()

In [ ]:
data <- pt_ora_res %>% tidyr::separate_rows(., geneID, sep="/") %>% dplyr::select(geneID, top_level) %>% dplyr::distinct() %>% dplyr::group_by(geneID) %>% dplyr::filter(n()<2) %>% 
    dplyr::group_by(top_level) %>% dplyr::summarise(n=n()) %>% dplyr::mutate(top_level=ifelse(n<5, "Other", as.character(top_level))) %>% 
    dplyr::group_by(top_level) %>% dplyr::summarise(n=sum(n)) %>% 
    dplyr::mutate(top_level=factor(top_level, levels=names(top_level_color))) %>% dplyr::arrange(top_level) %>% 
    dplyr::mutate(p=100*n/sum(n))

In [ ]:
pdf("result/figures/figure_2/pie_ora_ptcg_split_1_wt.pdf", width=3, height=1.5)

pie <- ggplot(data, aes(x="", y=p, fill=top_level)) +
  geom_col(width=1) +
  coord_polar(theta="y") +
    scale_fill_manual(values=top_level_color) + 
  theme_global_set(4) + theme_void() + guides(fill=guide_legend(ncol=1, override.aes=list(alpha=1, size=1.5), keywidth=0.25, keyheight=0.25, default.unit="cm"))

gridExtra::grid.arrange(
    
    pie %>% egg::set_panel_size(., width=unit(2, "cm"), height=unit(2, "cm"))
    
)

dev.off()

### Split 2

In [ ]:
pt_ora_res <- pt_ora_pa(names(genes[genes==2]), rownames(fitgam))
pt_ora_res <- parse_hierarchy(pt_ora_res) %>% dplyr::mutate(top_level=factor(top_level, levels=names(top_level_color))) %>% dplyr::filter(p.adjust<=0.05) %>% na.omit()

In [ ]:
pdf("result/figures/figure_2/ora_ptcg_split_2_wt.pdf", width=6, height=10)

category_n <- pt_ora_res %>% dplyr::group_by(top_level) %>% dplyr::summarise(n=n()) %>% dplyr::pull(n) /5
(pt_ora_pl_pa(pt_ora_res, color=color_mat_1[1], adj_pval_thr=0.05, range_max=2) + theme_global_set(4)) + ggh4x::force_panelsizes(total_width=unit(2.2, "cm"), rows=unit(category_n, "cm"))

dev.off()

In [ ]:
data <- pt_ora_res %>% tidyr::separate_rows(., geneID, sep="/") %>% dplyr::select(geneID, top_level) %>% dplyr::distinct() %>% dplyr::group_by(geneID) %>% dplyr::filter(n()<2) %>% 
    dplyr::group_by(top_level) %>% dplyr::summarise(n=n()) %>% dplyr::mutate(top_level=ifelse(n<5, "Other", as.character(top_level))) %>% 
    dplyr::group_by(top_level) %>% dplyr::summarise(n=sum(n)) %>% 
    dplyr::mutate(top_level=factor(top_level, levels=names(top_level_color))) %>% dplyr::arrange(top_level) %>% 
    dplyr::mutate(p=100*n/sum(n))

In [ ]:
pdf("result/figures/figure_2/pie_ora_ptcg_split_2_wt.pdf", width=3, height=1.5)

pie <- ggplot(data, aes(x="", y=p, fill=top_level)) +
  geom_col(width=1) +
  coord_polar(theta="y") +
    scale_fill_manual(values=top_level_color) + 
  theme_global_set(4) + theme_void() + guides(fill=guide_legend(ncol=1, override.aes=list(alpha=1, size=1.5), keywidth=0.25, keyheight=0.25, default.unit="cm"))

gridExtra::grid.arrange(
    
    pie %>% egg::set_panel_size(., width=unit(2, "cm"), height=unit(2, "cm"))
    
)

dev.off()

### Split 3

In [ ]:
pt_ora_res <- pt_ora_pa(names(genes[genes==3]), rownames(fitgam))
pt_ora_res <- parse_hierarchy(pt_ora_res) %>% dplyr::mutate(top_level=factor(top_level, levels=names(top_level_color))) %>% dplyr::filter(p.adjust<=0.05) %>% na.omit()

In [ ]:
pdf("result/figures/figure_2/ora_ptcg_split_3_wt.pdf", width=6, height=10)

category_n <- pt_ora_res %>% dplyr::group_by(top_level) %>% dplyr::summarise(n=n()) %>% dplyr::pull(n) /5
(pt_ora_pl_pa(pt_ora_res, color=color_mat_1[1], adj_pval_thr=0.05, range_max=2) + theme_global_set(4)) + ggh4x::force_panelsizes(total_width=unit(2.2, "cm"), rows=unit(category_n, "cm"))

dev.off()

In [ ]:
data <- pt_ora_res %>% tidyr::separate_rows(., geneID, sep="/") %>% dplyr::select(geneID, top_level) %>% dplyr::distinct() %>% dplyr::group_by(geneID) %>% dplyr::filter(n()<2) %>% 
    dplyr::group_by(top_level) %>% dplyr::summarise(n=n()) %>% dplyr::mutate(top_level=ifelse(n<5, "Other", as.character(top_level))) %>% 
    dplyr::group_by(top_level) %>% dplyr::summarise(n=sum(n)) %>% 
    dplyr::mutate(top_level=factor(top_level, levels=names(top_level_color))) %>% dplyr::arrange(top_level) %>% 
    dplyr::mutate(p=100*n/sum(n))

In [ ]:
pdf("result/figures/figure_2/pie_ora_ptcg_split_3_wt.pdf", width=3, height=1.5)

pie <- ggplot(data, aes(x="", y=p, fill=top_level)) +
  geom_col(width=1) +
  coord_polar(theta="y") +
    scale_fill_manual(values=top_level_color) + 
  theme_global_set(4) + theme_void() + guides(fill=guide_legend(ncol=1, override.aes=list(alpha=1, size=1.5), keywidth=0.25, keyheight=0.25, default.unit="cm"))

gridExtra::grid.arrange(
    
    pie %>% egg::set_panel_size(., width=unit(2, "cm"), height=unit(2, "cm"))
    
)

dev.off()

## PTPG D6 (WT) vs B (WT)

In [ ]:
pt_ora_res <- pt_ora_pa(ptpg[ptpg$ptpg_class %in% condition_vec_1[2], ][["gene"]], rownames(fitgam))
pt_ora_res <- parse_hierarchy(pt_ora_res) %>% dplyr::mutate(top_level=factor(top_level, levels=names(top_level_color))) %>% dplyr::filter(p.adjust<=0.05) %>% na.omit()

In [ ]:
pdf("result/figures/figure_2/ora_ptpg_d6_vs_ctr_wt.pdf", width=6, height=10)

category_n <- pt_ora_res %>% dplyr::group_by(top_level) %>% dplyr::summarise(n=n()) %>% dplyr::pull(n) /5
(pt_ora_pl_pa(pt_ora_res, color=color_mat_1[2], adj_pval_thr=0.05, range_max=2) + theme_global_set(4)) + ggh4x::force_panelsizes(total_width=unit(2.2, "cm"), rows=unit(category_n, "cm"))

dev.off()

In [ ]:
data <- pt_ora_res %>% tidyr::separate_rows(., geneID, sep="/") %>% dplyr::select(geneID, top_level) %>% dplyr::distinct() %>% dplyr::group_by(geneID) %>% dplyr::filter(n()<2) %>% 
    dplyr::group_by(top_level) %>% dplyr::summarise(n=n()) %>% dplyr::mutate(top_level=ifelse(n<5, "Other", as.character(top_level))) %>% 
    dplyr::group_by(top_level) %>% dplyr::summarise(n=sum(n)) %>% 
    dplyr::mutate(top_level=factor(top_level, levels=names(top_level_color))) %>% dplyr::arrange(top_level) %>% 
    dplyr::mutate(p=100*n/sum(n))

In [ ]:
pdf("result/figures/figure_2/pie_ora_ptpg_d6_vs_ctr_wt.pdf", width=3, height=1.5)

pie <- ggplot(data, aes(x="", y=p, fill=top_level)) +
  geom_col(width=1) +
  coord_polar(theta="y") +
    scale_fill_manual(values=top_level_color) + 
  theme_global_set(4) + theme_void() + guides(fill=guide_legend(ncol=1, override.aes=list(alpha=1, size=1.5), keywidth=0.25, keyheight=0.25, default.unit="cm"))

gridExtra::grid.arrange(
    
    pie %>% egg::set_panel_size(., width=unit(2, "cm"), height=unit(2, "cm"))
    
)

dev.off()